In [1]:
import numpy as np
import pandas as pd
import warnings


In [2]:
warnings.filterwarnings("ignore")

### GET THE DATASET


In [3]:
#we can use read_csv function with this file as well the by defualt the seperator is , that is the re
#reason it is termed as csv but in our case the file is tabl seperated so we just use sep as \t and read_CSV FUNCTION WILL WORK
df=pd.read_csv('u.data',sep='\t')

In [4]:
df.head()
#Hre the first row is considered as columns and hence it is not coreect

,196,242,3,881250949
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [5]:
column_names=["user id", "item_id" ,"rating" , "timestamp"]

In [6]:
df=pd.read_csv('u.data',sep='\t',names=column_names)

In [7]:
df.head()

,user id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
df.shape

(100000, 4)

In [9]:
df['user id'].nunique()   #Unique users

943

In [10]:
df['item_id'].nunique()   #Unique Movies

1682

As we have the item id  which corresponds to some movie so we require one more dataset which is u.item

In [11]:
movie_titles=pd.read_csv('u.item',sep='\|' ,header=None)

As in the readme file of movie lens it is clearly mention the column names of the u.item file

So we required only 1,2 columns as it is having movie id and title

In [12]:
movie_titles=movie_titles[[0,1]]

In [13]:
movie_titles.columns=[ "item_id", "movie title"]
movie_titles.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


#### Now we have 2 dataframes we need to merge the dataframe based on item_id  and get the new dataframe for that we use pandas merge function

In [14]:
df=pd.merge(df,movie_titles,on="item_id")

In [15]:
df.head()

,user id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


# Exploratory Data Analysis

###### First we need to find out the average rating of particular movies

In [16]:
#This is getting the movies whose average rating is from high to low  
df.groupby('movie title').mean()['rating'].sort_values(ascending=False)

movie title
They Made Me a Criminal (1939)                  5.0
Marlene Dietrich: Shadow and Light (1996)       5.0
Saint of Fort Washington, The (1993)            5.0
Someone Else's America (1995)                   5.0
Star Kid (1997)                                 5.0
                                               ... 
Eye of Vichy, The (Oeil de Vichy, L') (1993)    1.0
King of New York (1990)                         1.0
Touki Bouki (Journey of the Hyena) (1973)       1.0
Bloody Child, The (1996)                        1.0
Crude Oasis, The (1995)                         1.0
Name: rating, Length: 1664, dtype: float64

In [17]:
#This will show how many ratings  provided for particular movie

df.groupby('movie title').count()['rating'].sort_values(ascending=False)

movie title
Star Wars (1977)                      583
Contact (1997)                        509
Fargo (1996)                          508
Return of the Jedi (1983)             507
Liar Liar (1997)                      485
                                     ... 
Great Day in Harlem, A (1994)           1
Other Voices, Other Rooms (1997)        1
Good Morning (1971)                     1
Girls Town (1996)                       1
� k�ldum klaka (Cold Fever) (1994)      1
Name: rating, Length: 1664, dtype: int64

In [18]:
ratings=pd.DataFrame(df.groupby('movie title').mean()['rating'])
ratings["num of rating"]=pd.DataFrame(df.groupby('movie title').count()['rating'])

In [19]:
ratings.head()

,rating,num of rating
movie title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


###### We can not recommend a movie based on 1 or 2 raters  that is the reason num of rating is required 

# Createing movie Recommnedation

we need to create a mtrix first with columns as titles of the movies rows as user id which will describe the user has given what rating to the movie and if not given then it will print NAN

In [20]:

moviemat=df.pivot_table(index="user id",columns="movie title" ,values="rating")

In [21]:
moviemat

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
user id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ratings.sort_values('num of rating',ascending=False).head()

,rating,num of rating
movie title,,
Star Wars (1977),4.358491,583
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485


In [23]:
#movie wise user rating
starwars_user_ratings=moviemat['Star Wars (1977)']
starwars_user_ratings.head()

user id
1    5.0
2    5.0
3    NaN
4    5.0
5    4.0
Name: Star Wars (1977), dtype: float64

In [24]:
#Correlation of starwars with other movies

similar_to_startwars=moviemat.corrwith(starwars_user_ratings)
similar_to_startwars.head()

movie title
'Til There Was You (1997)    0.872872
1-900 (1994)                -0.645497
101 Dalmatians (1996)        0.211132
12 Angry Men (1957)          0.184289
187 (1997)                   0.027398
dtype: float64

###### Remove the NAN Values

In [25]:
corr_starwars=pd.DataFrame(similar_to_startwars,columns=['Correlation'])
corr_starwars.head()

,Correlation
movie title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [26]:
corr_starwars.dropna(inplace=True)

In [27]:
corr_starwars

,Correlation
movie title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
...,...
Young Guns (1988),0.186377
Young Guns II (1990),0.228615
"Young Poisoner's Handbook, The (1995)",-0.007374


### If you want to suggest any movie to someone we need to take the highest correlation and suggest based on that


In [28]:
corr_starwars.sort_values('Correlation',ascending=False)

,Correlation
movie title,
Hollow Reed (1996),1.0
Commandments (1997),1.0
Cosi (1996),1.0
No Escape (1994),1.0
Stripes (1981),1.0
...,...
Roseanna's Grave (For Roseanna) (1997),-1.0
For Ever Mozart (1996),-1.0
American Dream (1990),-1.0


##### There can be the possibility that the highky correlated movies have just been recommended by few peoples only so in that case we need to have some threshold like if greater than 100 users have rating a particular movie that movie only we have to consider

In [29]:
ratings.head()

,rating,num of rating
movie title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [30]:
corr_starwars.head()

,Correlation
movie title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [31]:
corr_starwars = corr_starwars.join(ratings['num of rating'])
corr_starwars.head()

,Correlation,num of rating
movie title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41


In [34]:
corr_starwars[corr_starwars['num of rating']>100].sort_values('Correlation',ascending=False)

,Correlation,num of rating
movie title,,
Star Wars (1977),1.000000,583
"Empire Strikes Back, The (1980)",0.747981,367
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Austin Powers: International Man of Mystery (1997),0.377433,130
...,...,...
"Edge, The (1997)",-0.127167,113
As Good As It Gets (1997),-0.130466,112
Crash (1996),-0.148507,128


# PREDICT FUNCTION

In [38]:
def predict_movie(moviename):
    movie_user_ratings=moviemat[moviename]
    similar_to_movie=moviemat.corrwith(movie_user_ratings)
    corr_movie=pd.DataFrame(similar_to_movie,columns=['Correlation'])
    corr_movie.dropna(inplace=True)
    corr_movie = corr_movie.join(ratings['num of rating'])
    prediction=corr_movie[corr_movie['num of rating']>100].sort_values('Correlation',ascending=False)
    
    return prediction

In [39]:
prediction=predict_movie("Titanic (1997)")

In [40]:
prediction.head()

,Correlation,num of rating
movie title,,
Titanic (1997),1.000000,350
"River Wild, The (1994)",0.497600,146
"Abyss, The (1989)",0.472103,151
Bram Stoker's Dracula (1992),0.443560,120
True Lies (1994),0.435104,208
